In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.text import *

In [3]:
PATH = Path('../../data/menu')
TMP_PATH = PATH/'tmp'
TMP_PATH.mkdir(exist_ok=True)
rawData = 'Dish.csv'
!ls '../../data/menu/tmp'

datasetX.pkl  dishCodes2.npy  wordDict2.pkl  wordList2.pkl
datasetY.pkl  dishCodes.npy   wordDict.pkl   wordList.pkl


In [4]:
dfRaw = pd.read_csv((PATH/rawData).open())

In [5]:
def getTokenIdx(rows):
    freqs = Counter(wd for name in rows for wd in name.split(' ') )
    itos = [key for key, count in freqs.most_common()]
    itos.insert(0, '_bos_')
    itos.insert(1, '_pad_')
    itos.insert(2, '_eos_')
    itos.insert(3, '_unk')
    stoi = collections.defaultdict(lambda:3, {v:k for k,v in enumerate(itos)})
    strCodes = [([stoi[i] for i in name.split(' ')]) for name in rows ]
    strCodes = np.array(strCodes)
    np.save(TMP_PATH/'dishCodes.npy', strCodes)
    pickle.dump(itos, open(TMP_PATH/'wordList.pkl','wb'))
    pickle.dump(dict(stoi), open(TMP_PATH/'wordDict.pkl','wb'))
    # freqFreq = Counter(count for key, count in freqs.most_common())
    # freqFreq.most_common()
    return strCodes, stoi, itos
# strCodes, stoi, itos = getTokenIdx(dfRaw.name)

In [6]:
def loadTokenIdx():
    strCodes = np.load(TMP_PATH/'dishCodes.npy')
    itos = pickle.load(open(TMP_PATH/'wordList.pkl','rb'))
    stoi = collections.defaultdict(lambda: 3, pickle.load(open(TMP_PATH/'wordDict.pkl','rb')))
    return strCodes, stoi, itos
strCodes, stoi, itos = loadTokenIdx()

### word vector

In [7]:
def dict2Vec(w2vDict):
    return np.random.rand(len(w2vDict), 30)
dict2Vec(stoi).shape

(147016, 30)

In [21]:
mostLength = int(np.percentile([len(o) for o in strCodes], 98))
mostLength

16

In [47]:
a = "1234567"
np.random.randint(10)

8

In [93]:
def genDataSet(strCodes, size, mostLength, genLength):
    Xs, Ys = [], []
    for i in range(size):
        randIdxes = random.sample(range(len(strCodes)), genLength)
        X, Y = [], []
        for randIdx in randIdxes:
            if len(strCodes[randIdx]) > mostLength:
                break
            X += strCodes[randIdx]
            Y += [0]*(len(strCodes[randIdx])-1)+[1]
        if np.sum(Y) == genLength:
            Xs.append(X)
            Ys.append(Y)
    pickle.dump(Xs, open(TMP_PATH/'datasetX.pkl','wb'))
    pickle.dump(Ys, open(TMP_PATH/'datasetY.pkl','wb'))
    return Xs, Ys
def loadDataSet():
    Xs = pickle.load(open(TMP_PATH/'datasetX.pkl','rb'))
    Ys = pickle.load(open(TMP_PATH/'datasetY.pkl','rb'))
    return Xs, Ys

# Xs, Ys = genDataSet(strCodes, 10000, mostLength, 5)
Xs, Ys = loadDataSet()
def interpret(Xs, Ys):
    XsOg = ""
    pick = np.random.randint(len(Xs))
    for i, x in enumerate(Xs[pick]):
        XsOg = XsOg + itos[x] + [' ', '|'][Ys[pick][i] == 1]
    print(len(Xs))
    print(XsOg)
interpret(Xs, Ys)

9111
Soup, Tomato, with Macaroni|Roast Saddle of Mutton, Red Currant Jelly, Cabbage, Potatoes and Rolls 9/-|GRANATINA (Mixed Flavors)|Frankfurters with Beans|Fresh Crab Legs|


In [95]:
def splitDataset(Xs, Ys):
    np.random.seed(9)
    Xs, Ys = np.array(Xs), np.array(Ys)
    isTrainIds = np.random.rand(len(Xs))>0.1
    trnXs, trnYs = Xs[isTrainIds], Ys[isTrainIds]
    valXs, valYs = Xs[~isTrainIds], Ys[~isTrainIds]
    pickle.dump(trnXs, open(TMP_PATH/'trnXs.pkl','wb'))
    pickle.dump(trnYs, open(TMP_PATH/'trnYs.pkl','wb'))
    pickle.dump(valXs, open(TMP_PATH/'valXs.pkl','wb'))
    pickle.dump(valYs, open(TMP_PATH/'valYs.pkl','wb'))
    return trnXs, trnYs, valXs, valYs

def loadSplitDataset():
    trnXs = pickle.load(open(TMP_PATH/'trnXs.pkl','rb'))
    trnYs = pickle.load(open(TMP_PATH/'trnYs.pkl','rb'))
    valXs = pickle.load(open(TMP_PATH/'valXs.pkl','rb'))
    valYs = pickle.load(open(TMP_PATH/'valYs.pkl','rb'))
    return trnXs, trnYs, valXs, valYs

# trnXs, trnYs, valXs, valYs = splitDataset(Xs, Ys)
trnXs, trnYs, valXs, valYs = loadSplitDataset()
interpret(trnXs, trnYs)
interpret(valXs, valYs)

8202
WIENER SCHNITZEL SPAETZLES, ONE VEG.|Canned Plums|De Lossy-Holden, Brut, '92 Champagne|Goose Liver au Naturel in its Jelly|Whiskey, Old Tom Pepper|
909
Borjumáj piritva burg|Chopped Beef Steak, Smothered Onions, Sliced Tomato|OUR SPECIAL GROUND HAMBURGER SANDWICH - Very Delicious|Pouding Chambord|Cold Fresh Salmon, Garni, Mayonnaise|


In [89]:
class Seq2SeqDataset(Dataset):
    def __init__(self, x, y): self.x,self.y = x,y
    def __getitem__(self, idx): return A(self.x[idx], self.y[idx])
    def __len__(self): return len(self.x)

In [98]:
trnDS = Seq2SeqDataset(trnXs, trnYs)
valDS = Seq2SeqDataset(valXs, valYs)

In [97]:
bs=125
trnSamp = SortishSampler(trnXs, key=lambda x: len(trnXs), bs=bs)
valSamp = SortSampler(valXs, key=lambda x:len(valXs))

In [101]:
trnDL = DataLoader(trnDS, bs,transpose=True, transpose_y=True, num_workers=1, 
                    pad_idx=1, pre_pad=False, sampler=trnSamp)
valDL = DataLoader(valDS, int(bs*1.6), transpose=True, transpose_y=True, num_workers=1, 
                    pad_idx=1, pre_pad=False, sampler=valSamp)

In [99]:
DataLoader??